# Homework Four:  MyPython 1.1 -- Completing the Expression Language and Adding Function Definitions

## General Homework Policies for CS 320

Please provide answers to these problems by filling in this notebook and submitting it via websubmit by Tuesday 2/20 by midnight. If the lab for this week involved a separate notebook, it is due at the same time. For any problems involving diagrams or drawings, you may try your hand at ASCII art or you may submit an electronic version of your drawings, either using drawing software or by scanning or photographing hand drawings. A message has been posted to Piazza showing you how to automatically include these in your notebook (a nice touch).  Make sure in the latter case that the result is legible; if we can not understand your drawing, it will receive no credit. 

There is a grace period, longer than 15 minutes but shorter than a couple of hours, after the deadlines, to account for temporarily problems with submission (dropped connections etc.). In cases where I have uploaded
a solution late on Wednesday night (probably close to the 15 minute mark!) no submissions will be accepted after the solution is uploaded. 

You may submit by Wednesday midnight for a 10% penalty; after that no credit will be given. 

I will drop the lowest homework/lab combination at the end of the term.

For coding problems any <i> reasonable </i> solution, in which you understand the spirit of the exercise and try to learn this week's material through your efforts, will be graded based solely on the test cases. In general, shortcuts to avoid learning the material will be unreasonable. For example, you may not use any Python libraries or functions that make your task dramatically easier (e.g., you may use split(), replace(...), int(...), and float(...), as mentioned below, and any other normal functions, but may NOT use the Python library for regular expressions).  Good Python style, using list comprehensions, Numpy, creating helper functions in addition to what is required, etc., <i>when appropriate,</i> are quite reasonable! Check with me if you have questions about this. 
<p>
<span style="color:red"><b>Lab problems</b> may be worked on collaboratively at the lab and afterwards, with no need to indicate whom you collaborated with. (Of course, if you simply copy, then you will not learn what you need for homeworks and tests, and your grade will reflect your lack of effort.)</span> 

<span style="color:red"><b>Homework problems</b> must be solved individually, and will be subject to the College rules on plagiarism. <b>In particular, you are absolutely forbidden to copy code or answers from another student or from a source other than my slides and materials I provide to you.</b> We will run the code from notebooks through an electronic service that checks for plagiarism, and any violation will be dealt with harshly, up to and including an automatic drop of one letter grade at the end of the term (this policy will be familiar to those who took CS 111 at BU). Significant offences will be referred to the College Academic Misconduct Committee. These policies are necessary to provide a fair environment for you to do your best work with the expectation that the quality and quantity of your efforts will be rewarded appropriately. <b>Please take this as seriously as I do.</b></span> 

<b> Make sure to click Cell -> Run All before submitting notebooks!!</b>

## Changes to the basic encoding of tokens

The encoding of token categories as integers is what is done in real compilers and interpreters 
when necessary, for efficiency, but is becoming unworkable as the grammar gets larger. 
I provided a token_to_string function to pretty-print the tokens, e.g., in the ASTs, but still it
is a pain. 

Therefore going forward from this homework on, we will simply use strings to indicate the token category.
Names for these are provided, as you can see in the next code cell, but I almost always just use the
string itself. This may take a minute or two to understand, since it is different from the previous 
homeworks, but the benefits in readability should become obvious once you get going on the homework: the 
names for the tokens are identical to the syntax of the tokens, 'id', 'int', '>=', 'lambda', etc.:
<pre>
          ('id', 'x')     ('int', 5)      ('>=',)      ('lambda',)
</pre>
You can see how this plays out in the provided code. 


### Lexical analyzer code. DO NOT MODIFY!

In [1]:

# Token Categories -- slightly different than in previous homeworks, strings rather than integers.  

# NOTE: YOU DO NOT NEED TO USE THESE, BETTER TO USE THE STRINGS THEMSELVES, BUT PROVIDED HERE
# IN CASE USED IN OLD CODE

ident = 'id'          # used in token as (indent, <string> )    
integer = 'int'       #  (integer, <value> )
floating = 'float'    #  (floating, <value> )  
trueTok = 'True'
falseTok = 'False'
noneTok = "None"
plus = '+'            #  (plus,)    rest as this one, no attribute necessary
minus = '-'          
mult = '*'
div = '/' 
mod = '%'
exp = '**'
lparen = '('
rparen = ')'
lbrace = '['
rbrace = ']'
equals = '=='
assign = '='
colon = ':'
lt = '<'
gt = '>'
ge = '>='
le = '<='
ne = '!='
andTok = 'and'
orTok = 'or'
notTok = 'not'
defTok = 'def'
semicolon = ';'
comma = ','
let = 'let'
inTok = 'in'
ifTok = 'if'
thenTok = 'then'
elseTok = 'else'
lambdaTok = 'lambda'
apply = 'apply'
returnTok = 'return'
error = 'error'       #  (error, <string> )     gives spelling of token that caused the lexical error

# Non-terminals are encoded as pairs using these codes:

S = 'S'             # (S,)  etc. 
E = 'E'
A = 'A'
L = 'L'
T = 'T'
F = 'F'
I = 'I'
C = 'C'
Top = 'Top'
Bor = 'Bor'
Band = 'Band'
Bnot = 'Bnot'


# special token for end of input

end_of_input = '$'       # (end_of_input,) will be pretty-printed as ($,)

def tokenToString(t):
    if t == None:
        return str(t)
    elif t[0] in ['int','float','id']:
        return '(' + t[0] + ',' + str(t[1]) + ')'
    else:
        return '(' + t[0] + ',)'
        
def tokenListToString(lst):
    res = '[ '
    for t in lst[:-1]:
        res = res + tokenToString(t) + ', '
    res = res + tokenToString(lst[-1]) + ' ]'
    return res

In [2]:

# Code for lexer

# put white space between each separator or operator token and split into words

def splitTokens(s):
    for t in ['+','-','*','/','(',')','[',']',',','<', ':', '>', '=', '!','%']:
        s = s.replace(t,' ' + t + ' ')
    # now repair two-character tokens
    s = s.replace('<  =','<=')
    s = s.replace('>  =','>=')
    s = s.replace('=  =','==')
    s = s.replace('!  =','!=')
    s = s.replace('*  *','**')
    return s.split()

def isLetter(c):
    return 'a' <= c <= 'z' or 'A' <= c <= 'Z'

def isDigit(c):
    return '0' <= c <= '9' 

def isIdToken(s):
    state = 1
    for ch in s:
        if state == 1:
            if isLetter(ch) or ch == '_':
                state = 2
            else:
                return False
        elif state == 2:
            if isLetter(ch) or isDigit(ch) or ch == '_':
                state = 2
            else:
                return False
    return (state == 2)

                        
def isIntToken(s):
    if s == '0':
        return True
    state = 1
    for ch in s:
        if state == 1:
            if isDigit(ch) and ch != '0':
                state = 2
            else:
                return False
        elif state == 2:
            if isDigit(ch):
                state = 2
            else:
                return False
    return (state == 2)
    
def isFloatToken(s):
    state = 1
    finalStates = [4,6]
    for ch in s:
        if state == 1:
            if isDigit(ch) and ch != '0':
                state = 2
            elif ch == '0':
                state = 3
            elif ch == '.':
                state = 5
            else:
                return False
        elif state == 2:
            if isDigit(ch):
                state = 2
            elif ch == '.':
                state = 4
            else:
                return False
        elif state == 3:
            if ch == '.':
                state = 4
            else:
                return False
        elif state == 4:
            if isDigit(ch):
                state = 4
            else:
                return False
        elif state == 5:
            if isDigit(ch):
                state = 6
            else:
                return False
        elif state == 6:
            if isDigit(ch):
                state = 6
            else:
                return False                
                
    return (state in finalStates)   
    
def isAssignToken(s):
    return s == '='

def isPlusToken(s):
    return s == '+'
    
def isMinusToken(s):
    return s == '-'

def isMultToken(s):
    return s == '*'
    
def isDivToken(s):
    return s == '/'
    
def isModToken(s):
    return s == '%'

def isExpToken(s):
    return s == '**'

def isEqualsToken(s):
    return s == '=='

def isNEToken(s):
    return s == '!='

def isLEToken(s):
    return s == '<='
    
def isGEToken(s):
    return s == '>='

def isLTToken(s):
    return s == '<'
    
def isGTToken(s):
    return s == '>'

def isLparenToken(s):
    return s == '('
    
def isRparenToken(s):
    return s == ')'

def isLbraceToken(s):
    return s == '['
    
def isRbraceToken(s):
    return s == ']'

def isColonToken(s):
    return s == ':'

def isSemicolonToken(s):
    return s == ';'

def isCommaToken(s):
    return s == ','

def isLetToken(s):
    return s == 'let'

def isLambdaToken(s):
    return s == 'lambda'

def isInToken(s):
    return s == 'in'

def isIfToken(s):
    return s == 'if'

def isThenToken(s):
    return s == 'then'

def isElseToken(s):
    return s == 'else'

def isDefToken(s):
    return s == 'def'

def isAndToken(s):
    return s == 'and'

def isOrToken(s):
    return s == 'or'

def isNotToken(s):
    return s == 'not'

def isReturnToken(s):
    return s == 'return'

def isTrueToken(s):
    return s == 'True'

def isFalseToken(s):
    return s == 'False'

def isNoneToken(s):
    return s == 'None'

def stringToToken(t):
    if isPlusToken(t):
        return (plus,) 
    elif isMinusToken(t):
        return (minus,) 
    elif isMultToken(t):
        return (mult,) 
    elif isDivToken(t):
        return (div,) 
    elif isExpToken(t):
        return (exp,) 
    elif isModToken(t):
        return (mod,) 
    elif isLTToken(t):
        return (lt,) 
    elif isGTToken(t):
        return (gt,) 
    elif isGEToken(t):
        return (ge,)    
    elif isLEToken(t):
        return (le,) 
    elif isEqualsToken(t):
        return (equals,)
    elif isNEToken(t):
        return (ne,)     
    elif isAndToken(t):
        return (andTok,) 
    elif isOrToken(t):
        return (orTok,)
    elif isNotToken(t):
        return (notTok,)    
    elif isLparenToken(t):
        return (lparen,) 
    elif isRparenToken(t):
        return (rparen,)
    elif isLbraceToken(t):
        return (rbrace,) 
    elif isRbraceToken(t):
        return (rparen,)
    elif isDefToken(t):
        return (defTok,)    
    elif isAssignToken(t):
        return (assign,)
    elif isColonToken(t):
        return (colon,)
    elif isSemicolonToken(t):
        return (semicolon,)
    elif isCommaToken(t):
        return (comma,)
    elif isLetToken(t):
        return (let,)
    elif isLambdaToken(t):
        return (lambdaTok,)
    elif isIfToken(t):
        return (ifTok,)
    elif isThenToken(t):
        return (thenTok,)
    elif isElseToken(t):
        return (elseTok,)
    elif isInToken(t):
        return (inTok,)
    elif isReturnToken(t):
        return (returnTok,)
    elif isTrueToken(t):
        return (trueTok,)
    elif isFalseToken(t):
        return (falseTok,)
    elif isNoneToken(t):
        return (noneTok,)
    elif isIdToken(t):
        return (ident,t)
    elif isIntToken(t):
        return (integer,int(t)) 
    elif isFloatToken(t):
        return (floating,float(t)) 

    else:
        return (error,)


def lexer(s):
    return [stringToToken(t) for t in splitTokens(s)]

#lexer('2 * 3 /def:in x * (if 3.0 * 7 then 9 else x)and(or**plus/))*,]{{%}')

In [3]:
# Utilities for grammars

def rule_to_string(r):
    s = r[0][0] + ' := '
    for t in r[1]:
        s = s + t[0] + ' '
    return s
                          
def pprint_grammar(G): 
    for k in range(len(G)):
        print(str(k) + ": " + rule_to_string(G[k])) 
  
Grammar1_1 =    [(('S',),(('Top',),)),                       #  S -> I 
         (('Top',),(('A',),)),                       #  I -> A
         (('Top',),(('I',),)),                       #  I -> E 
         (('Top',),(('def',),('id',),('(',),('id',),(')',),(':',),('return',),('I',))),   #  I -> def id ( id ) : return I 
         (('A',),(('id',),('=',),('I',))),          #  A -> id = L
         (('I',),(('if',),('Bor',),('then',),('I',),('else',),('I',))), #  A -> id = L
         (('I',),(('lambda',),('id',),(':',),('I',))), # L -> lambda x : Bor 
         (('I',),(('let',),('A',),('in',),('I',))), 
         (('I',),(('Bor',),)),                      # L -> Bor 
         (('Bor',),(('Bor',),('or',),('Band',))),   #  Bor -> Bor or Band
         (('Bor',),(('Band',),)),                   #  Bor -> Band
         (('Band',),(('Band',),('and',),('Bnot',))),#  Band -> Band and Bnot
         (('Band',),(('Bnot',),)),                  #  Band -> Bnot 
         (('Bnot',),(('not',),('Bnot',))),          #  Bnot -> not Bnot
         (('Bnot',),(('C',),)),                     #  Bnot -> C 
         (('C',),(('E',),('==',),('E',))),          #  C -> E == E    (not allowing multiple comparisonss)
         (('C',),(('E',),('!=',),('E',))),          #  C -> E != E 
         (('C',),(('E',),('<',),('E',))),           #  C -> E < E 
         (('C',),(('E',),('<=',),('E',))),          #  C -> E <= E 
         (('C',),(('E',),('>',),('E',))),           #  C -> E > E 
         (('C',),(('E',),('>=',),('E',))),          #  C -> E >= E 
         (('C',),(('E',))),                         #  C -> E 
         (('E',),(('E',),('+',),('T',))),          #  E -> E + T 
         (('E',),(('E',),('-',),('T',))),          #  E -> E - T 
         (('E',),(('T',),)),                       #  E -> T
         (('T',),(('T',),('*',),('F',))),          #  T -> T * F
         (('T',),(('T',),('/',),('F',))),          #  T -> T / F
         (('T',),(('T',),('%',),('F',))),          #  T -> T % F
         (('T',),(('F',),)),                       #  T -> F
         (('F',),(('-',),('F',))),                 #  F -> - F
         (('F',),(('F1',),)),                       #  F -> F1
         (('F1',),(('F2',), ('**',),('F1',))),     #  F1 -> F2 ** F1     Right-associates!
         (('F1',),(('F2',),)),                      #  F1 -> F2                                                              
         (('F2',),(  ('F2',), ('(',), ('I',),(')',)) ),
         (('F2',),(('id',),)),                      #  F2 -> id
         (('F2',),(('F3',),)), 
         (('F3',),(('int',),) ),                    #  F2 -> integer
         (('F3',),(('float',),)),                   #  F2 -> floating
         (('F3',),(('True',),)), 
         (('F3',),(('False',),)), 
         (('F3',),(('(',),('I',),(')',)) )  ]        #  F2 -> ( E )       

#pprint_grammar(Grammar1_1)

#   Parser 

### The next two cells contain provided code for the homework; DO NOT MODIFY!

Changes from the last time are the table for the new grammar, in which the columns are indexed by strings and now err = None and Accept = 0. You should
not have to know this, but just in case....


In [4]:
# DFA to drive the shift-reduce parser

# The parser DFA will tell you what to do when the stack is
# in a given configuration.  A positive number gives the
# state transition, a non-positive number k is a reduction by the
# rule -k, i.e., -4 means a reduction by rule 4 (t -> F) with no
# advance in the inputListOfTokens string.  None is an error and
# 0 is accept. 

err = None
accept = 0

table = {
  0:{'I': 1, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 4, '%': None, ':': None, 'False': 5, 'then': None, 'Top': 6, 'id': 7, '-': 8, 'A': 9, '+': None, 'T': 10, 'in': None, 'if': 11, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 16, 'let': 17, 'F1': 18, 'Band': 19, '$': None, '!=': None, 'Bor': 20, '<': None, 'int': 21, 'def': 22, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 24, 'lambda': 25},
  1:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -2, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  2:{'I': None, 'F': None, '==': -28, '=': None, 'and': -28, 'float': None, 'else': -28, 'E': None, '%': -28, ':': None, 'False': None, 'then': -28, 'Top': None, 'id': None, '-': -28, 'A': None, '+': -28, 'T': None, 'in': -28, 'if': None, '>=': -28, 'or': -28, '<=': -28, '/': -28, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -28, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -28, '!=': -28, 'Bor': None, '<': -28, 'int': None, 'def': None, '*': -28, 'True': None, '>': -28, '**': None, 'Bnot': None, 'lambda': None},
  3:{'I': None, 'F': None, '==': -37, '=': None, 'and': -37, 'float': None, 'else': -37, 'E': None, '%': -37, ':': None, 'False': None, 'then': -37, 'Top': None, 'id': None, '-': -37, 'A': None, '+': -37, 'T': None, 'in': -37, 'if': None, '>=': -37, 'or': -37, '<=': -37, '/': -37, 'F3': None, 'return': None, 'S': None, '(': -37, 'not': None, 'C': None, ')': -37, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -37, '!=': -37, 'Bor': None, '<': -37, 'int': None, 'def': None, '*': -37, 'True': None, '>': -37, '**': -37, 'Bnot': None, 'lambda': None},
  4:{'I': None, 'F': None, '==': 27, '=': None, 'and': -21, 'float': None, 'else': -21, 'E': None, '%': None, ':': None, 'False': None, 'then': -21, 'Top': None, 'id': None, '-': 30, 'A': None, '+': 31, 'T': None, 'in': -21, 'if': None, '>=': 33, 'or': -21, '<=': 26, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -21, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -21, '!=': 28, 'Bor': None, '<': 29, 'int': None, 'def': None, '*': None, 'True': None, '>': 32, '**': None, 'Bnot': None, 'lambda': None},
  5:{'I': None, 'F': None, '==': -39, '=': None, 'and': -39, 'float': None, 'else': -39, 'E': None, '%': -39, ':': None, 'False': None, 'then': -39, 'Top': None, 'id': None, '-': -39, 'A': None, '+': -39, 'T': None, 'in': -39, 'if': None, '>=': -39, 'or': -39, '<=': -39, '/': -39, 'F3': None, 'return': None, 'S': None, '(': -39, 'not': None, 'C': None, ')': -39, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -39, '!=': -39, 'Bor': None, '<': -39, 'int': None, 'def': None, '*': -39, 'True': None, '>': -39, '**': -39, 'Bnot': None, 'lambda': None},
  6:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': 0, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  7:{'I': None, 'F': None, '==': -34, '=': 34, 'and': -34, 'float': None, 'else': -34, 'E': None, '%': -34, ':': None, 'False': None, 'then': -34, 'Top': None, 'id': None, '-': -34, 'A': None, '+': -34, 'T': None, 'in': -34, 'if': None, '>=': -34, 'or': -34, '<=': -34, '/': -34, 'F3': None, 'return': None, 'S': None, '(': -34, 'not': None, 'C': None, ')': -34, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -34, '!=': -34, 'Bor': None, '<': -34, 'int': None, 'def': None, '*': -34, 'True': None, '>': -34, '**': -34, 'Bnot': None, 'lambda': None},
  8:{'I': None, 'F': 35, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': None, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  9:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -1, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  10:{'I': None, 'F': None, '==': -24, '=': None, 'and': -24, 'float': None, 'else': -24, 'E': None, '%': 40, ':': None, 'False': None, 'then': -24, 'Top': None, 'id': None, '-': -24, 'A': None, '+': -24, 'T': None, 'in': -24, 'if': None, '>=': -24, 'or': -24, '<=': -24, '/': 38, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -24, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -24, '!=': -24, 'Bor': None, '<': -24, 'int': None, 'def': None, '*': 39, 'True': None, '>': -24, '**': None, 'Bnot': None, 'lambda': None},
  11:{'I': None, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 41, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': 19, '$': None, '!=': None, 'Bor': 42, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 24, 'lambda': None},
  12:{'I': None, 'F': None, '==': -35, '=': None, 'and': -35, 'float': None, 'else': -35, 'E': None, '%': -35, ':': None, 'False': None, 'then': -35, 'Top': None, 'id': None, '-': -35, 'A': None, '+': -35, 'T': None, 'in': -35, 'if': None, '>=': -35, 'or': -35, '<=': -35, '/': -35, 'F3': None, 'return': None, 'S': None, '(': -35, 'not': None, 'C': None, ')': -35, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -35, '!=': -35, 'Bor': None, '<': -35, 'int': None, 'def': None, '*': -35, 'True': None, '>': -35, '**': -35, 'Bnot': None, 'lambda': None},
  13:{'I': 44, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 41, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': 11, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 37, 'let': 17, 'F1': 18, 'Band': 19, '$': None, '!=': None, 'Bor': 45, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 24, 'lambda': 25},
  14:{'I': None, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 41, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 46, 'lambda': None},
  15:{'I': None, 'F': None, '==': None, '=': None, 'and': -14, 'float': None, 'else': -14, 'E': None, '%': None, ':': None, 'False': None, 'then': -14, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': -14, 'if': None, '>=': None, 'or': -14, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -14, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -14, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  16:{'I': None, 'F': None, '==': -32, '=': None, 'and': -32, 'float': None, 'else': -32, 'E': None, '%': -32, ':': None, 'False': None, 'then': -32, 'Top': None, 'id': None, '-': -32, 'A': None, '+': -32, 'T': None, 'in': -32, 'if': None, '>=': -32, 'or': -32, '<=': -32, '/': -32, 'F3': None, 'return': None, 'S': None, '(': 47, 'not': None, 'C': None, ')': -32, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -32, '!=': -32, 'Bor': None, '<': -32, 'int': None, 'def': None, '*': -32, 'True': None, '>': -32, '**': 48, 'Bnot': None, 'lambda': None},
  17:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': 49, '-': None, 'A': 50, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  18:{'I': None, 'F': None, '==': -30, '=': None, 'and': -30, 'float': None, 'else': -30, 'E': None, '%': -30, ':': None, 'False': None, 'then': -30, 'Top': None, 'id': None, '-': -30, 'A': None, '+': -30, 'T': None, 'in': -30, 'if': None, '>=': -30, 'or': -30, '<=': -30, '/': -30, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -30, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -30, '!=': -30, 'Bor': None, '<': -30, 'int': None, 'def': None, '*': -30, 'True': None, '>': -30, '**': None, 'Bnot': None, 'lambda': None},
  19:{'I': None, 'F': None, '==': None, '=': None, 'and': 51, 'float': None, 'else': -10, 'E': None, '%': None, ':': None, 'False': None, 'then': -10, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': -10, 'if': None, '>=': None, 'or': -10, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -10, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -10, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  20:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': -8, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': -8, 'if': None, '>=': None, 'or': 52, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -8, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -8, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  21:{'I': None, 'F': None, '==': -36, '=': None, 'and': -36, 'float': None, 'else': -36, 'E': None, '%': -36, ':': None, 'False': None, 'then': -36, 'Top': None, 'id': None, '-': -36, 'A': None, '+': -36, 'T': None, 'in': -36, 'if': None, '>=': -36, 'or': -36, '<=': -36, '/': -36, 'F3': None, 'return': None, 'S': None, '(': -36, 'not': None, 'C': None, ')': -36, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -36, '!=': -36, 'Bor': None, '<': -36, 'int': None, 'def': None, '*': -36, 'True': None, '>': -36, '**': -36, 'Bnot': None, 'lambda': None},
  22:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': 53, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  23:{'I': None, 'F': None, '==': -38, '=': None, 'and': -38, 'float': None, 'else': -38, 'E': None, '%': -38, ':': None, 'False': None, 'then': -38, 'Top': None, 'id': None, '-': -38, 'A': None, '+': -38, 'T': None, 'in': -38, 'if': None, '>=': -38, 'or': -38, '<=': -38, '/': -38, 'F3': None, 'return': None, 'S': None, '(': -38, 'not': None, 'C': None, ')': -38, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -38, '!=': -38, 'Bor': None, '<': -38, 'int': None, 'def': None, '*': -38, 'True': None, '>': -38, '**': -38, 'Bnot': None, 'lambda': None},
  24:{'I': None, 'F': None, '==': None, '=': None, 'and': -12, 'float': None, 'else': -12, 'E': None, '%': None, ':': None, 'False': None, 'then': -12, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': -12, 'if': None, '>=': None, 'or': -12, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -12, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -12, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  25:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': 54, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  26:{'I': None, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 55, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  27:{'I': None, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 56, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  28:{'I': None, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 57, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  29:{'I': None, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 58, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  30:{'I': None, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': None, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 59, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  31:{'I': None, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': None, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 60, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  32:{'I': None, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 61, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  33:{'I': None, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 62, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  34:{'I': 63, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 41, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': 11, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 37, 'let': 17, 'F1': 18, 'Band': 19, '$': None, '!=': None, 'Bor': 45, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 24, 'lambda': 25},
  35:{'I': None, 'F': None, '==': -29, '=': None, 'and': -29, 'float': None, 'else': -29, 'E': None, '%': -29, ':': None, 'False': None, 'then': -29, 'Top': None, 'id': None, '-': -29, 'A': None, '+': -29, 'T': None, 'in': -29, 'if': None, '>=': -29, 'or': -29, '<=': -29, '/': -29, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -29, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -29, '!=': -29, 'Bor': None, '<': -29, 'int': None, 'def': None, '*': -29, 'True': None, '>': -29, '**': None, 'Bnot': None, 'lambda': None},
  36:{'I': None, 'F': None, '==': -34, '=': None, 'and': -34, 'float': None, 'else': -34, 'E': None, '%': -34, ':': None, 'False': None, 'then': -34, 'Top': None, 'id': None, '-': -34, 'A': None, '+': -34, 'T': None, 'in': -34, 'if': None, '>=': -34, 'or': -34, '<=': -34, '/': -34, 'F3': None, 'return': None, 'S': None, '(': -34, 'not': None, 'C': None, ')': -34, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -34, '!=': -34, 'Bor': None, '<': -34, 'int': None, 'def': None, '*': -34, 'True': None, '>': -34, '**': -34, 'Bnot': None, 'lambda': None},
  37:{'I': None, 'F': None, '==': -32, '=': None, 'and': -32, 'float': None, 'else': -32, 'E': None, '%': -32, ':': None, 'False': None, 'then': -32, 'Top': None, 'id': None, '-': -32, 'A': None, '+': -32, 'T': None, 'in': -32, 'if': None, '>=': -32, 'or': -32, '<=': -32, '/': -32, 'F3': None, 'return': None, 'S': None, '(': 47, 'not': None, 'C': None, ')': -32, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -32, '!=': -32, 'Bor': None, '<': -32, 'int': None, 'def': None, '*': -32, 'True': None, '>': -32, '**': 48, 'Bnot': None, 'lambda': None},
  38:{'I': None, 'F': 64, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': None, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  39:{'I': None, 'F': 65, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': None, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  40:{'I': None, 'F': 66, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': None, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  41:{'I': None, 'F': None, '==': 27, '=': None, 'and': -21, 'float': None, 'else': -21, 'E': None, '%': None, ':': None, 'False': None, 'then': -21, 'Top': None, 'id': None, '-': 30, 'A': None, '+': 31, 'T': None, 'in': -21, 'if': None, '>=': 33, 'or': -21, '<=': 26, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -21, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -21, '!=': 28, 'Bor': None, '<': 29, 'int': None, 'def': None, '*': None, 'True': None, '>': 32, '**': None, 'Bnot': None, 'lambda': None},
  42:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': 67, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': 52, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  43:{'I': None, 'F': None, '==': -24, '=': None, 'and': -24, 'float': None, 'else': -24, 'E': None, '%': 40, ':': None, 'False': None, 'then': -24, 'Top': None, 'id': None, '-': -24, 'A': None, '+': -24, 'T': None, 'in': -24, 'if': None, '>=': -24, 'or': -24, '<=': -24, '/': 38, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -24, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -24, '!=': -24, 'Bor': None, '<': -24, 'int': None, 'def': None, '*': 39, 'True': None, '>': -24, '**': None, 'Bnot': None, 'lambda': None},
  44:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': 68, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  45:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': -8, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': -8, 'if': None, '>=': None, 'or': 52, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -8, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -8, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  46:{'I': None, 'F': None, '==': None, '=': None, 'and': -13, 'float': None, 'else': -13, 'E': None, '%': None, ':': None, 'False': None, 'then': -13, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': -13, 'if': None, '>=': None, 'or': -13, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -13, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -13, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  47:{'I': 69, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 41, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': 11, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 37, 'let': 17, 'F1': 18, 'Band': 19, '$': None, '!=': None, 'Bor': 45, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 24, 'lambda': 25},
  48:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': None, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': None, 'C': None, ')': None, 'F2': 37, 'let': None, 'F1': 70, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  49:{'I': None, 'F': None, '==': None, '=': 34, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  50:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': 71, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  51:{'I': None, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 41, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 72, 'lambda': None},
  52:{'I': None, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 41, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 37, 'let': None, 'F1': 18, 'Band': 73, '$': None, '!=': None, 'Bor': None, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 24, 'lambda': None},
  53:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': 74, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  54:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': 75, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  55:{'I': None, 'F': None, '==': None, '=': None, 'and': -18, 'float': None, 'else': -18, 'E': None, '%': None, ':': None, 'False': None, 'then': -18, 'Top': None, 'id': None, '-': 30, 'A': None, '+': 31, 'T': None, 'in': -18, 'if': None, '>=': None, 'or': -18, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -18, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -18, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  56:{'I': None, 'F': None, '==': None, '=': None, 'and': -15, 'float': None, 'else': -15, 'E': None, '%': None, ':': None, 'False': None, 'then': -15, 'Top': None, 'id': None, '-': 30, 'A': None, '+': 31, 'T': None, 'in': -15, 'if': None, '>=': None, 'or': -15, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -15, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -15, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  57:{'I': None, 'F': None, '==': None, '=': None, 'and': -16, 'float': None, 'else': -16, 'E': None, '%': None, ':': None, 'False': None, 'then': -16, 'Top': None, 'id': None, '-': 30, 'A': None, '+': 31, 'T': None, 'in': -16, 'if': None, '>=': None, 'or': -16, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -16, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -16, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  58:{'I': None, 'F': None, '==': None, '=': None, 'and': -17, 'float': None, 'else': -17, 'E': None, '%': None, ':': None, 'False': None, 'then': -17, 'Top': None, 'id': None, '-': 30, 'A': None, '+': 31, 'T': None, 'in': -17, 'if': None, '>=': None, 'or': -17, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -17, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -17, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  59:{'I': None, 'F': None, '==': -23, '=': None, 'and': -23, 'float': None, 'else': -23, 'E': None, '%': 40, ':': None, 'False': None, 'then': -23, 'Top': None, 'id': None, '-': -23, 'A': None, '+': -23, 'T': None, 'in': -23, 'if': None, '>=': -23, 'or': -23, '<=': -23, '/': 38, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -23, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -23, '!=': -23, 'Bor': None, '<': -23, 'int': None, 'def': None, '*': 39, 'True': None, '>': -23, '**': None, 'Bnot': None, 'lambda': None},
  60:{'I': None, 'F': None, '==': -22, '=': None, 'and': -22, 'float': None, 'else': -22, 'E': None, '%': 40, ':': None, 'False': None, 'then': -22, 'Top': None, 'id': None, '-': -22, 'A': None, '+': -22, 'T': None, 'in': -22, 'if': None, '>=': -22, 'or': -22, '<=': -22, '/': 38, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -22, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -22, '!=': -22, 'Bor': None, '<': -22, 'int': None, 'def': None, '*': 39, 'True': None, '>': -22, '**': None, 'Bnot': None, 'lambda': None},
  61:{'I': None, 'F': None, '==': None, '=': None, 'and': -19, 'float': None, 'else': -19, 'E': None, '%': None, ':': None, 'False': None, 'then': -19, 'Top': None, 'id': None, '-': 30, 'A': None, '+': 31, 'T': None, 'in': -19, 'if': None, '>=': None, 'or': -19, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -19, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -19, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  62:{'I': None, 'F': None, '==': None, '=': None, 'and': -20, 'float': None, 'else': -20, 'E': None, '%': None, ':': None, 'False': None, 'then': -20, 'Top': None, 'id': None, '-': 30, 'A': None, '+': 31, 'T': None, 'in': -20, 'if': None, '>=': None, 'or': -20, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -20, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -20, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  63:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': -4, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -4, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  64:{'I': None, 'F': None, '==': -26, '=': None, 'and': -26, 'float': None, 'else': -26, 'E': None, '%': -26, ':': None, 'False': None, 'then': -26, 'Top': None, 'id': None, '-': -26, 'A': None, '+': -26, 'T': None, 'in': -26, 'if': None, '>=': -26, 'or': -26, '<=': -26, '/': -26, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -26, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -26, '!=': -26, 'Bor': None, '<': -26, 'int': None, 'def': None, '*': -26, 'True': None, '>': -26, '**': None, 'Bnot': None, 'lambda': None},
  65:{'I': None, 'F': None, '==': -25, '=': None, 'and': -25, 'float': None, 'else': -25, 'E': None, '%': -25, ':': None, 'False': None, 'then': -25, 'Top': None, 'id': None, '-': -25, 'A': None, '+': -25, 'T': None, 'in': -25, 'if': None, '>=': -25, 'or': -25, '<=': -25, '/': -25, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -25, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -25, '!=': -25, 'Bor': None, '<': -25, 'int': None, 'def': None, '*': -25, 'True': None, '>': -25, '**': None, 'Bnot': None, 'lambda': None},
  66:{'I': None, 'F': None, '==': -27, '=': None, 'and': -27, 'float': None, 'else': -27, 'E': None, '%': -27, ':': None, 'False': None, 'then': -27, 'Top': None, 'id': None, '-': -27, 'A': None, '+': -27, 'T': None, 'in': -27, 'if': None, '>=': -27, 'or': -27, '<=': -27, '/': -27, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -27, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -27, '!=': -27, 'Bor': None, '<': -27, 'int': None, 'def': None, '*': -27, 'True': None, '>': -27, '**': None, 'Bnot': None, 'lambda': None},
  67:{'I': 76, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 41, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': 11, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 37, 'let': 17, 'F1': 18, 'Band': 19, '$': None, '!=': None, 'Bor': 45, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 24, 'lambda': 25},
  68:{'I': None, 'F': None, '==': -40, '=': None, 'and': -40, 'float': None, 'else': -40, 'E': None, '%': -40, ':': None, 'False': None, 'then': -40, 'Top': None, 'id': None, '-': -40, 'A': None, '+': -40, 'T': None, 'in': -40, 'if': None, '>=': -40, 'or': -40, '<=': -40, '/': -40, 'F3': None, 'return': None, 'S': None, '(': -40, 'not': None, 'C': None, ')': -40, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -40, '!=': -40, 'Bor': None, '<': -40, 'int': None, 'def': None, '*': -40, 'True': None, '>': -40, '**': -40, 'Bnot': None, 'lambda': None},
  69:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': 77, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  70:{'I': None, 'F': None, '==': -31, '=': None, 'and': -31, 'float': None, 'else': -31, 'E': None, '%': -31, ':': None, 'False': None, 'then': -31, 'Top': None, 'id': None, '-': -31, 'A': None, '+': -31, 'T': None, 'in': -31, 'if': None, '>=': -31, 'or': -31, '<=': -31, '/': -31, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -31, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -31, '!=': -31, 'Bor': None, '<': -31, 'int': None, 'def': None, '*': -31, 'True': None, '>': -31, '**': None, 'Bnot': None, 'lambda': None},
  71:{'I': 78, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 41, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': 11, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 37, 'let': 17, 'F1': 18, 'Band': 19, '$': None, '!=': None, 'Bor': 45, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 24, 'lambda': 25},
  72:{'I': None, 'F': None, '==': None, '=': None, 'and': -11, 'float': None, 'else': -11, 'E': None, '%': None, ':': None, 'False': None, 'then': -11, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': -11, 'if': None, '>=': None, 'or': -11, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -11, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -11, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  73:{'I': None, 'F': None, '==': None, '=': None, 'and': 51, 'float': None, 'else': -9, 'E': None, '%': None, ':': None, 'False': None, 'then': -9, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': -9, 'if': None, '>=': None, 'or': -9, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -9, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -9, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  74:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': 79, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  75:{'I': 80, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 41, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': 11, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 37, 'let': 17, 'F1': 18, 'Band': 19, '$': None, '!=': None, 'Bor': 45, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 24, 'lambda': 25},
  76:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': 81, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  77:{'I': None, 'F': None, '==': -33, '=': None, 'and': -33, 'float': None, 'else': -33, 'E': None, '%': -33, ':': None, 'False': None, 'then': -33, 'Top': None, 'id': None, '-': -33, 'A': None, '+': -33, 'T': None, 'in': -33, 'if': None, '>=': -33, 'or': -33, '<=': -33, '/': -33, 'F3': None, 'return': None, 'S': None, '(': -33, 'not': None, 'C': None, ')': -33, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -33, '!=': -33, 'Bor': None, '<': -33, 'int': None, 'def': None, '*': -33, 'True': None, '>': -33, '**': -33, 'Bnot': None, 'lambda': None},
  78:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': -7, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': -7, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -7, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -7, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  79:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': 82, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  80:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': -6, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': -6, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -6, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -6, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  81:{'I': 83, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 41, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': 11, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 37, 'let': 17, 'F1': 18, 'Band': 19, '$': None, '!=': None, 'Bor': 45, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 24, 'lambda': 25},
  82:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': 84, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  83:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': -5, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': -5, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': -5, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -5, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  84:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': 85, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': None, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None},
  85:{'I': 86, 'F': 2, '==': None, '=': None, 'and': None, 'float': 3, 'else': None, 'E': 41, '%': None, ':': None, 'False': 5, 'then': None, 'Top': None, 'id': 36, '-': 8, 'A': None, '+': None, 'T': 43, 'in': None, 'if': 11, '>=': None, 'or': None, '<=': None, '/': None, 'F3': 12, 'return': None, 'S': None, '(': 13, 'not': 14, 'C': 15, ')': None, 'F2': 37, 'let': 17, 'F1': 18, 'Band': 19, '$': None, '!=': None, 'Bor': 45, '<': None, 'int': 21, 'def': None, '*': None, 'True': 23, '>': None, '**': None, 'Bnot': 24, 'lambda': 25},
  86:{'I': None, 'F': None, '==': None, '=': None, 'and': None, 'float': None, 'else': None, 'E': None, '%': None, ':': None, 'False': None, 'then': None, 'Top': None, 'id': None, '-': None, 'A': None, '+': None, 'T': None, 'in': None, 'if': None, '>=': None, 'or': None, '<=': None, '/': None, 'F3': None, 'return': None, 'S': None, '(': None, 'not': None, 'C': None, ')': None, 'F2': None, 'let': None, 'F1': None, 'Band': None, '$': -3, '!=': None, 'Bor': None, '<': None, 'int': None, 'def': None, '*': None, 'True': None, '>': None, '**': None, 'Bnot': None, 'lambda': None}
}



def action(lst,table):
    state = 0
    for token in lst:
        try:
            state = table[state][token[0]]
        except KeyError:
            return err          
        if state == err:        # error state is implicit so fail as soon as you hit an error
            return err
    return state



### Reminder: You should not modify anything above this line unless you check with Professor Snyder first!
<pre>




</pre>

## Problem One: MyPython 1.1 ASTs  (10 pts)

As mentioned in lecture on Thursday, the task for this week is to add to MyPython:
<ul>
    <li> the rest of the arithmetic operators, 
    <li> boolean operators and constants and the relational operators,
    <li> conditional if-then-else expressions (in a more familiar form than the absurd syntax Python uses),
    <li> lambda expressions and appplications, and
    <li> syntactic sugar for function definitions.
</ul>
The new grammar is below, 40 rules! (The parser DFA has 86 states.)

Your job for this week is to add code to generate the AST for the augmented set of features. If you understood last week's homework, you should be able to do this quickly, with a lot of cutting and pasting. 

<span style="color:red"><b>Note carefully that the way tokens are represented is somewhat different, with strings instead of integers. You can see by the provided code what is necessary to do. Should be easier once you understand!</b></span>

Most of what you need to know can be gleaned from the test examples. Here are some additional hints:
<ul>
    <li> The additional arithmetic and boolean operators are implemented by analogy by the standard ones. Note that exponentiation ** is <it>right-associative</it>, but this is taken care of by the grammar, and you should not have to worry about this while creating the ASTs, nor while writing the evaluator. 
    <li> We have added something new to our language, namely, the boolean literals <code>True</code> and <code>False</code>. Basically you treat these the same as identifiers, integers, and floats, but with no corresponding value in the tuple, and they becomes leaves in the AST.  The solution ASTs (posted on the web site) should clarify. 
    <li> The other cases are for the lambda, if statement (which is different from the Python (i.e., we will use  "if x < 3 then 5 else 6" instead of the yucky "5 if x < 3 else 6"), and application. The solutions should clarify. 

</ul>
For deep background, the grammar was created after I stared at the following precedence chart for several hours:

http://www.mathcs.emory.edu/~valerie/courses/fall10/155/resources/op_precedence.html


In [5]:
print("\nGrammar for MyPython 1.1:\n")
pprint_grammar(Grammar1_1)


Grammar for MyPython 1.1:

0: S := Top 
1: Top := A 
2: Top := I 
3: Top := def id ( id ) : return I 
4: A := id = I 
5: I := if Bor then I else I 
6: I := lambda id : I 
7: I := let A in I 
8: I := Bor 
9: Bor := Bor or Band 
10: Bor := Band 
11: Band := Band and Bnot 
12: Band := Bnot 
13: Bnot := not Bnot 
14: Bnot := C 
15: C := E == E 
16: C := E != E 
17: C := E < E 
18: C := E <= E 
19: C := E > E 
20: C := E >= E 
21: C := E 
22: E := E + T 
23: E := E - T 
24: E := T 
25: T := T * F 
26: T := T / F 
27: T := T % F 
28: T := F 
29: F := - F 
30: F := F1 
31: F1 := F2 ** F1 
32: F1 := F2 
33: F2 := F2 ( I ) 
34: F2 := id 
35: F2 := F3 
36: F3 := int 
37: F3 := float 
38: F3 := True 
39: F3 := False 
40: F3 := ( I ) 


In [6]:
# Code for shift-reduce parser

# pretty-printing parser configurations

def pprintParser(parsingStack,inputListOfTokens):
    totalWidth = 80
    smallestGap = 6
    largestStackLength = int(totalWidth*0.6 - smallestGap/2)   # most characters to display
    largestInputLength = totalWidth - largestStackLength - smallestGap
    
    p = '| '
    for symbol in parsingStack:
        p = p + tokenToString(symbol) + ' '
    if len(p) > largestStackLength:
        ind = int(len(p) - largestStackLength + 9)
        p = '| ... ' + p[ind:]
        
    q = ""
    for tok in inputListOfTokens[:-1]:
        q = q + tokenToString(tok) + ' '
    if len(inputListOfTokens) > 0:
        q = q + tokenToString(inputListOfTokens[-1]) 

    if len(q) > largestInputLength:
        ind = int(largestInputLength - 9)
        q = q[:ind] + ' ... ' 
    
    q = q + ' |'
        
    p = p + (' ' * (totalWidth - len(p) - len(q)))
    print(p+q)
    
# Pretty-printing code for testing your solution

def ASTToString(t):
    if t[0] in ['int','id','float','True', 'False']:
        return str(t)
    elif len(t) == 2:
        return '(' + t[0] + ',' + ASTToString(t[1]) + ')'
    elif len(t) == 3:
        return '(' + t[0] + ',' + ASTToString(t[1]) + ',' + ASTToString(t[2]) + ')'
    elif len(t) == 4:
        return '(' + t[0] + ',' + ASTToString(t[1]) + ',' + ASTToString(t[2]) + ',' + ASTToString(t[3]) + ')'
    #else
    print("** " + str(len(t)))
    print("** " + str(t))
    return "ERROR!"
   
def pprintAST(t):
    pprintASTHelper(t,'')

def pprintASTHelper(t,indent):
    if (t[0] in [ident,integer,floating]):
        print(indent + '(' + t[0] + ',' + str(t[1]) + ')')
    else:
        print(indent + t[0])
        for k in range(1,len(t)):
            pprintASTHelper(t[k],indent + '\t')
    

# parse takes a list of tokens and determines if it is a 
# well-formed arithmetic expression.
# This version will also build an AST

def parse(list_of_input_tokens,rules,table,verbose=False):
    
    # add end of input marker
    list_of_input_tokens.append( (end_of_input,) )
    
    # input stack; use pop(0) to remove front item in list
    input_stack = list_of_input_tokens
    
    # parsing stack; use append to push onto end and pop() to pop from end
    parsing_stack = []
    
    if verbose:
        print('Input Tokens: ' + tokenListToString(input_stack) + '\n')
        pprintParser(parsing_stack,input_stack) 
    
    while( len(input_stack) > 0 ):
        
        # Now we will "lookahead" at the next symbol on the input and ask the automaton what to do
        # a positive number means to shift, and a negative number -n means to reduce by rule n
        
        n = action(parsing_stack+[input_stack[0]],table)
        if n == accept:   # reduce by start rule, success
            if verbose:
                print("\nAccept!")
            return parsing_stack[-1][1] 
        elif n == err:     #  problem on stack!
            if verbose:
                print("\nERROR: No transition here:")
                pprintParser(parsing_stack,input_stack)
            return None

        elif n > 0:     # shift 
            if verbose:
                print('\naction: ' + str(n) + '\tshift\n')
            parsing_stack.append( input_stack.pop(0)) 
        else:         # reduce by rule -n 
            if verbose:
                print('\naction: ' + str(n) + '\treduce by rule ' + str(-n) + ': ' + rule_to_string(rules[-n])+'\n')
            r = -n
            LHS = rules[r][0]
            if r in [0,1,2,8,10,12,14,21,24,28,30,32,35]:  # unary rules 1, 2, 7, 10 just pass ast up 
                (_,ast) = parsing_stack.pop()
                parsing_stack.append( (LHS[0],ast) )
            elif r in [34,36,37,38,39]:                  # rules 34,36,37 for values create leaf node by passing tuple up
                t = parsing_stack.pop()                  # 38, 39 are constants True and False
                parsing_stack.append( (LHS[0],t) )
            elif r in [22,23,25,26,27,15,16,17,18,19,20,9,11,31]:   # binary operators -- add the rest!
                (_,ast2) = parsing_stack.pop()
                (operator,) = parsing_stack.pop()
                (_,ast1) = parsing_stack.pop()
                parsing_stack.append( (LHS[0],(operator,ast1,ast2)) )
            elif r in [29, 13]:                        #   unary operators -- add the not!
                (_,ast) = parsing_stack.pop()      # F  
                (operator,) = parsing_stack.pop()  # - 
                parsing_stack.append( (LHS[0],(operator,ast)) )
            elif r == 4:                           # A := id = I   
                (_,ast2) = parsing_stack.pop()     # I
                (operator,) = parsing_stack.pop()  # =
                t = parsing_stack.pop()            # id
                parsing_stack.append( (LHS[0],(operator,t,ast2)) )
            elif r == 7:                           # I := let A in I  
                (_,ast2) = parsing_stack.pop()     # I
                parsing_stack.pop()                # in 
                (_,ast1) = parsing_stack.pop()     # A 
                (operator,) = parsing_stack.pop()  # let 
                parsing_stack.append( (LHS[0],(operator,ast1,ast2)) )
            elif r == 40:                          # F3 := ( E )
                parsing_stack.pop()                # (
                (_,ast) = parsing_stack.pop()      # E
                parsing_stack.pop()                # )
                parsing_stack.append( (LHS[0],ast) )
            elif r ==6:
                (_,ast2) = parsing_stack.pop()
                parsing_stack.pop()
                id1 = parsing_stack.pop()
                (lam,) = parsing_stack.pop()
                parsing_stack.append((LHS[0], (lam,id1,ast2))) 
            elif r == 33:
                parsing_stack.pop()                # (
                (_,ast) = parsing_stack.pop()      # E
                parsing_stack.pop()                # )
                (_,ast2) = parsing_stack.pop()
                parsing_stack.append( (LHS[0],(apply,ast2,ast)) )
            elif r == 5:
                (_,ast) = parsing_stack.pop()  
                parsing_stack.pop()  
                (_,ast1) = parsing_stack.pop() 
                parsing_stack.pop() 
                (_,ast2) = parsing_stack.pop()
                (if1,) = parsing_stack.pop()
                parsing_stack.append((LHS[0], (if1,ast2,ast1, ast))) 
            elif r == 3:
                (_,ast) = parsing_stack.pop()  
                parsing_stack.pop()  
                parsing_stack.pop()  
                parsing_stack.pop()  
                t =parsing_stack.pop() 
                parsing_stack.pop()  
                t1 = parsing_stack.pop() 
                parsing_stack.pop() 
                parsing_stack.append((LHS[0], (assign,t1,(lambdaTok,t, ast))))
            else:
                print("parse stack error: action " + str(n))
                return None
   
        if verbose:
            pprintParser(parsing_stack,input_stack)
            
    return None     # this will never be executed

def testParserAST(s,verbose=False):
    t = parse(lexer(s),Grammar1_1,table,verbose)
    if t == None or t == False:
        print("Error: No AST generated!")
    else:
        print('Input: ' + str(s))
        print("\nAbstract Syntax Tree:\n")
        print(ASTToString(t)) 
        print()
        pprintAST(t)

In [7]:
# Test 1     Notice that ** is right-associative
s = '-x * y + 2 / 3 ** 4 ** (5-y)' 
testParserAST(s) 

Input: -x * y + 2 / 3 ** 4 ** (5-y)

Abstract Syntax Tree:

(+,(*,(-,('id', 'x')),('id', 'y')),(/,('int', 2),(**,('int', 3),(**,('int', 4),(-,('int', 5),('id', 'y'))))))

+
	*
		-
			(id,x)
		(id,y)
	/
		(int,2)
		**
			(int,3)
			**
				(int,4)
				-
					(int,5)
					(id,y)


In [8]:
# Test 2
s = 'True and False or not (2 < 4) and x > 2'
testParserAST(s) 

Input: True and False or not (2 < 4) and x > 2

Abstract Syntax Tree:

(or,(and,('True',),('False',)),(and,(not,(<,('int', 2),('int', 4))),(>,('id', 'x'),('int', 2))))

or
	and
		True
		False
	and
		not
			<
				(int,2)
				(int,4)
		>
			(id,x)
			(int,2)


In [9]:
# Test 3
s = 'x != 4 and 3 == 3.0 or x <= 1 and 8.2 <= flag'
testParserAST(s) 

Input: x != 4 and 3 == 3.0 or x <= 1 and 8.2 <= flag

Abstract Syntax Tree:

(or,(and,(!=,('id', 'x'),('int', 4)),(==,('int', 3),('float', 3.0))),(and,(<=,('id', 'x'),('int', 1)),(<=,('float', 8.2),('id', 'flag'))))

or
	and
		!=
			(id,x)
			(int,4)
		==
			(int,3)
			(float,3.0)
	and
		<=
			(id,x)
			(int,1)
		<=
			(float,8.2)
			(id,flag)


In [10]:
# Test 4
s = 'lambda x : lambda y : lambda z: x + y + z'
testParserAST(s) 

Input: lambda x : lambda y : lambda z: x + y + z

Abstract Syntax Tree:

(lambda,('id', 'x'),(lambda,('id', 'y'),(lambda,('id', 'z'),(+,(+,('id', 'x'),('id', 'y')),('id', 'z')))))

lambda
	(id,x)
	lambda
		(id,y)
		lambda
			(id,z)
			+
				+
					(id,x)
					(id,y)
				(id,z)


In [11]:
# Test 5
s = 'x = f(h(g(f(x))))'
testParserAST(s) 

Input: x = f(h(g(f(x))))

Abstract Syntax Tree:

(=,('id', 'x'),(apply,('id', 'f'),(apply,('id', 'h'),(apply,('id', 'g'),(apply,('id', 'f'),('id', 'x'))))))

=
	(id,x)
	apply
		(id,f)
		apply
			(id,h)
			apply
				(id,g)
				apply
					(id,f)
					(id,x)


In [12]:
# Test 6
s = 'f = let x = f(g(2)) in lambda y : f(y ** x)'
testParserAST(s) 

Input: f = let x = f(g(2)) in lambda y : f(y ** x)

Abstract Syntax Tree:

(=,('id', 'f'),(let,(=,('id', 'x'),(apply,('id', 'f'),(apply,('id', 'g'),('int', 2)))),(lambda,('id', 'y'),(apply,('id', 'f'),(**,('id', 'y'),('id', 'x'))))))

=
	(id,f)
	let
		=
			(id,x)
			apply
				(id,f)
				apply
					(id,g)
					(int,2)
		lambda
			(id,y)
			apply
				(id,f)
				**
					(id,y)
					(id,x)


In [13]:
# Test 7
s = 'g = (lambda x : (let y = x * 2 in (lambda z : (let x = (lambda f : f(x)) in x(y)))))'
testParserAST(s) 

Input: g = (lambda x : (let y = x * 2 in (lambda z : (let x = (lambda f : f(x)) in x(y)))))

Abstract Syntax Tree:

(=,('id', 'g'),(lambda,('id', 'x'),(let,(=,('id', 'y'),(*,('id', 'x'),('int', 2))),(lambda,('id', 'z'),(let,(=,('id', 'x'),(lambda,('id', 'f'),(apply,('id', 'f'),('id', 'x')))),(apply,('id', 'x'),('id', 'y')))))))

=
	(id,g)
	lambda
		(id,x)
		let
			=
				(id,y)
				*
					(id,x)
					(int,2)
			lambda
				(id,z)
				let
					=
						(id,x)
						lambda
							(id,f)
							apply
								(id,f)
								(id,x)
					apply
						(id,x)
						(id,y)


In [14]:
# Test 8
s = 'if x then if y then 1 else 2 else if y < 1 then 1 else 3'
testParserAST(s) 

Input: if x then if y then 1 else 2 else if y < 1 then 1 else 3

Abstract Syntax Tree:

(if,('id', 'x'),(if,('id', 'y'),('int', 1),('int', 2)),(if,(<,('id', 'y'),('int', 1)),('int', 1),('int', 3)))

if
	(id,x)
	if
		(id,y)
		(int,1)
		(int,2)
	if
		<
			(id,y)
			(int,1)
		(int,1)
		(int,3)


In [15]:
# Test 9
s = 'def abs(x): return if x < 0 then -x else x'
testParserAST(s) 

Input: def abs(x): return if x < 0 then -x else x

Abstract Syntax Tree:

(=,('id', 'abs'),(lambda,('id', 'x'),(if,(<,('id', 'x'),('int', 0)),(-,('id', 'x')),('id', 'x'))))

=
	(id,abs)
	lambda
		(id,x)
		if
			<
				(id,x)
				(int,0)
			-
				(id,x)
			(id,x)


In [16]:
# Test 10
s = "def select_function(x): return if x == 0 then lambda x : x else if x == 1 then lambda x : lambda y : y else lambda z : z"

testParserAST(s) 

Input: def select_function(x): return if x == 0 then lambda x : x else if x == 1 then lambda x : lambda y : y else lambda z : z

Abstract Syntax Tree:

(=,('id', 'select_function'),(lambda,('id', 'x'),(if,(==,('id', 'x'),('int', 0)),(lambda,('id', 'x'),('id', 'x')),(if,(==,('id', 'x'),('int', 1)),(lambda,('id', 'x'),(lambda,('id', 'y'),('id', 'y'))),(lambda,('id', 'z'),('id', 'z'))))))

=
	(id,select_function)
	lambda
		(id,x)
		if
			==
				(id,x)
				(int,0)
			lambda
				(id,x)
				(id,x)
			if
				==
					(id,x)
					(int,1)
				lambda
					(id,x)
					lambda
						(id,y)
						(id,y)
				lambda
					(id,z)
					(id,z)


## Problem Two: AST Evaluator for the new Grammar (10 pts)


For this problem you must add code to the eval function to process the new operators, the boolean literals, the lambda expression, and applications. As with the last problem, the code below is the solution for HW 03, modified for the new encoding of tokens categories as strings. Here are some hints:
<ul>
<li> The additional arithmetic operators (%, **) will be processed similarly to the usual +, -, etc. 
<li> The boolean constant <code>True</code> should evaluate to the integer 1, and <code>False</code> should evaluate to 0; 
<li> The boolean operators (<code>or</code> , <code>and</code>, <code>not</code>) and the relational operators ( <, >, <=, >=, ==, and != ) should evaluate using their Python counterparts, but then you must explicitly convert the resulting Python boolean  to 1 for <code>True</code> and 0 for <code>False</code>, i.e., 
<pre>     res =  < evaluate the boolean or relational operator to a Python boolean >
          return 1 if res else 0 
</pre>
Python would of course return <code>True</code> and <code>False</code> as the result of the boolean operators and the relational tests, but since we have no type checking, we can not distinguish between integers representing booleans and normal integers. We will fix this when we add dynamic type checking in a future assignment.  
<li> lambda expressions evaluate to themselves (i.e., you don't do anything, just return them).
<li> Applications must have a lambda expression as the first argument and any expression as the second, for example, 
<pre>
           ('apply', ('lambda', ('id', 'x'),  body  ),  argument  )
</pre>
for two expressions <code>body</code> and <code>argument</code>.
To evaluate this, you must recursively evaluate <code>argument</code> in the environment (without changing it) to
obtain a value $n$, and then evaluate <code>body</code> in an environment to which the binding ( 'x', $n$ ) has
been added. You can pretty much use your code from the <code>let</code> expression from last time, with minimal changes! <span style="color:red">In order to avoid the pathologies discussed (sort of) in lecture, we will for the present only allow functions of one argument, that is, the body of a lambda expression can NOT be another lambda. So you do not need to use closures--we will put them in next time!
<p>We will also not worry about type checking for expressions that make no sense, such as 5(4). Again, we'll do this soon....</span> 


</ul>

In [17]:
## Evaluator for ASTs:      

# eval(t,env,memory) takes an abstract syntax tree, an environment, and a global memory and evaluates
# the term in the context of the bindings in env, which is a stack, so
# the first occurrence of the binding is the one that is used; if a variable
# is not found in the env, it is looked for in the global memory; if not there
# an unbound variable exception is raised

# global memory is a dictionary of bindings, e.g., { 'x': 4.5, 'y':-1 }

# bindings are in the form of a tuple (str,value), e.g., ('x',4.5) so
# environment is a list of bindings, e.g., [ ('x',4.5), ('y',-1), ('x',4) ]

# for efficiency, env stack grows from left to right, so push binding on env for
# the recursive call by using env + [binding]

# Use the following for errors with unbound variables

class UnboundVariableException(Exception):
    def __init__(self, name):
        self.name = name
        
# To raise an exception involving a variable (ident, s), use   raise UnboundVariableException(s) 
# Don't worry about catching exceptions, we will do that in the next problem


# look up a variable id (the string representation, not the whole token) in the
# environment from end of list to beginning; if not there, look in memory, if
# not there, raise an exception:   raise UnboundVariableException(name) 

def lookup(name,env,memory):
    for i in range(len(env)-1,-1,-1):
        if(name == env[i][0]):
            return env[i][1]
    if name in memory:
        return memory[name]
    else:
        raise UnboundVariableException(name)       

def eval(t,env,memory):
    if t[0] == '+':
        return eval(t[1],env,memory) + eval(t[2],env,memory)
    elif t[0] == '*':
        return eval(t[1],env,memory) * eval(t[2],env,memory)
    elif t[0] == '**':
        return eval(t[1],env,memory) ** eval(t[2],env,memory)
    elif t[0] == '%':
        return eval(t[1],env,memory) % eval(t[2],env,memory)
    elif t[0] == '-' and len(t) == 3:
        return eval(t[1],env,memory) - eval(t[2],env,memory)
    elif t[0] == '-' and len(t) == 2:
        return - eval(t[1],env,memory)
    elif t[0] == '/':
        return eval(t[1],env,memory) / eval(t[2],env,memory)
    elif t[0] == 'True':
        return 1;
    elif t[0] == 'False':
        return 0;
    elif t[0] in ['or', 'and', 'not', '<', '>', '<=', '>=','==', '!=']:
        res = eval(t[1],env,memory)
        if res:
            return 1
        else:
            return 0
    elif t[0] == 'if':
        if(eval(t[1],env,memory)):
            return eval(t[2],env,memory)
        else:
            return eval(t[3],env,memory)
            
    elif t[0] == 'id':                   # identifier, so must look up in env then global memory
        return lookup(t[1],env,memory)
    elif t[0] == '=':                    # assignment expression
        val = eval(t[2],env,memory)
        memory[t[1][1]] = val
        return val
    elif t[0] == 'let':                  # let expression, so must extend env with new binding
        binding = (t[1][1][1],eval(t[1][2],env,memory))
        return eval(t[2],env + [binding],memory)
    elif t[0] == 'apply':                  # let expression, so must extend env with new binding
        f = eval(t[1], env, memory)
        (_,(_,name), body) = f
        binding = (name,eval(t[2],env,memory))
        #binding2 = (t[1][1][1], eval(t[1][2],env + [binding],memory))
        return eval(body,env + [binding],memory)
    elif t[0] == 'lambda':
        return t
    else:   # int or float
        return t[1]



In [18]:
# test 0 
s = '1 < 9'
t = parse(lexer(s),Grammar1_1,table)
print("Should be 1:")
print(eval(t,[],{ 'z' : 2, 'x' : 1 })) 

Should be 1:
1


In [19]:
# test 2 
s = '9 > 6'
t = parse(lexer(s),Grammar1_1,table)
print("Should be 1:")
print(eval(t,[],{})) 

Should be 1:
1


In [20]:
# test 2 
s = '(lambda x : x) (3)'
t = parse(lexer(s),Grammar1_1,table)
print("Should be 3:")
print(eval(t,[],{})) 

Should be 3:
3


In [21]:
# test 3 
s = '(lambda x : x * 2) ((lambda x : x + 1) (3))'
t = parse(lexer(s),Grammar1_1,table)
print("Should be 8:")
print(eval(t,[],{})) 

Should be 8:
8


In [22]:
# test 4 
s = '(lambda x : x * 2) ((lambda x : x + 1) ((let y = 4 in ((lambda x : x * x)(y)))))'
t = parse(lexer(s),Grammar1_1,table)
print("Should be 34:")
print(eval(t,[],{})) 

Should be 34:
34


In [23]:
# test 5 
s = '(lambda x : x * 2) ((lambda x : x + 1) (3))'
t = parse(lexer(s),Grammar1_1,table)
print("Should be 8:")
print(eval(t,[],{})) 

Should be 8:
8


In [24]:
# test 5 
s = 'if False then  2 else if True then 3 else 4'
t = parse(lexer(s),Grammar1_1,table)
print("Should be 3:")
print(eval(t,[],{})) 

Should be 3:
3


In [ ]:
# test 6 
s = 'let x = 4 in if False then  2 else if False then 3 else x'
t = parse(lexer(s),Grammar1_1,table)
print("Should be 4:")
print(eval(t,[],{})) 

Should be 4:
4


## Problem 3 (5 pts)

This is the interactive console from HW 03, unchanged. You can now try out your code and see if it works as expected. Don't skip this, as the graders will test out your code this way, and you want to know if it works!! 
<p> Just try all the basic things and verify that it works as you expect. Just don't use nested lambdas or lambda and let combined. 


In [ ]:
# Basic Interpreter

# use the following as a framework for imitating what I provide in the solution pdf
# to catch an exception, use something like this:
#            try:
#                print("Out["+str(lineNum)+"]: " + str(eval(t,[])))
#            except UnboundVariableException as n:
#                print("Unbound Variable: " + str(n))

lineNum = 1

global_memory = { }

s = input("\nMyPython 1.1\nInput equation or type \"quit\" to quit:\n\nIn ["+str(lineNum)+"]: ")

while True:
    if s == "":
        pass
    elif s == 'quit':      
        print("\nBye!")
        break
    elif s == "global":
        print(global_memory)
    else:
        t = parse(lexer(s), Grammar1_1, table, False)
        
        if t == None:
            print("\nError, try again!")
        else:
            try:
                print("Out["+str(lineNum)+"]: " + str(eval(t,[],global_memory)))
            except UnboundVariableException as n:
                print("Unbound Variable: " + str(n))

    lineNum += 1
    s = input("\nIn ["+str(lineNum)+"]: ")

  